In [1]:
import pickle
import numpy as np
from classes_colors import *
from sensor_cls import SensorCls
import plotly.graph_objects as go

In [2]:
with open("sensor_data_filtered.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

In [13]:
day, mat, sensor = 1, 1, 1
s = SensorCls(day, mat, sensor, sensor_data, labels, interp_funcs)
cls_list = s.get_interpolated_data()
cls_list_forced = s.get_interpolated_data(force_num_samples=100)

In [10]:
def plot_cls(cls_list, day, mat, sensor):
    title = f"Day {day}, Matrix {mat}, Sensor {sensor}"
    fig = go.Figure()

    times = np.array([])
    datas = [np.array([])] * 10

    for cls in cls_list:
        times = np.append(times, np.append(cls["sample_times"], cls["sample_times"][-1]+1))
        for i in range(10):
            datas[i] = np.append(datas[i], np.append(cls["sample_vals"][i], None))

    for i in range(10):
        fig.add_trace(go.Scatter(x=times,
                                 y=datas[i],
                                 mode="lines",
                                 name=f"H.S. {i}"))
    
    for cls in cls_list:
        fig.add_vrect(x0=cls["start"],
                      x1=cls["end"],
                      annotation_text=classes[cls["class"]],
                      annotation_position="top left",
                      fillcolor=colors[cls["class"]],
                      opacity=0.2,
                      line_width=0)

    fig.update_layout(title=title, title_x=0.5, width=1000, height=600)

    fig.update_yaxes(type="log")
    fig.show()

In [14]:
plot_cls(cls_list, day, mat, sensor)

In [15]:
plot_cls(cls_list_forced, day, mat, sensor)